* アンサンブル（ensemble):予測器のグループ
* アンサンブル学習（ensemble learning):一群の予測器のよそくを1つにまとめる学習
* アンサンブルメソッド（ensemble method):アンサンブル学習アルゴリズム
* ランダムフォレスト（random forest):訓練セットかr無作為に作った様々なサブセットを使って一連の決定木分類器を訓練し、予測するときにはすべての木の予測を集め、多数決で全体の予測クラスを決める
* ハード投票（hard voting)分類器：各分類器の予測を集め、多数決で決まったクラスを全体の予測とする
* 弱学習器(weak learner: 無作為な推測よりもわずか程度に良い）を十分集め、それぞれが十分多種多様なら、アンサンブルは強学習器（strong learner:高い正解率を達成する）になる
* 大数の法則（law of large numbers)：すべての分類器が完全に独立していて、誤りに相関関係がない場合に成り立つ

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=10000, noise=0.25)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC(probability = True)

voting_clf = VotingClassifier(
    estimators = [('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')
voting_clf.fit(X_train, y_train)

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
heads_proba = 0.51
coin_tosses = (np.random.rand(10000,10)< heads_proba).astype(np.int32)#51%の確率で1となるようにする
cumulative_heads_ratio = np.cumsum(coin_tosses, axis=0)/np.arange(1,10001).reshape(-1,1)#毎回どれくらいの割合表だったかを計算
plt.figure(figsize=(8,3.5))
plt.plot(cumulative_heads_ratio)
plt.plot([0,10000], [0.51,0.51], "k--", linewidth=2, label="51%")
plt.plot([0,10000], [0.5,0.5], "k-", linewidth=2, label="50%")
plt.xlabel("Number of coin tosses")
plt.ylabel("Heads ratio")
plt.legend(loc="lower right")
plt.axis([0, 10000, 0.42, 0.58])
plt.show()

* ソフト投票(soft voting)：すべての分類器がクラスに属する確率を推計できるメソッドを持つ場合、個別の分類器が推計する確率を平均し、もっとも確率の高いクラスを予測クラスとして返す

訓練セットから無作為に別々のサブセットをサンプリングして訓練する
* バギング（bagging: bootstrap aggreating)：サンプリングが重複あり
* ペースティング（pasting):サンプリングが重複なし
すべての分類器を予測したら、アンサンブルは単純にすべての予測器の予測を集計して新インスタンスに対する予測をする。\
集計関数
* 統計モード（statistical mode：予測の最頻値をとる）→分類
* 平均→回帰

異なるCPUコアや異なるサーバーを使ってすべての分類器を並列に訓練でき、予測も並列に実行できる！

個々の予測器が訓練に使うサブセットの多様性はバギングのほうがペースティングより高い→バイアスが高くなる\
いっぽうで、予測器の相関が下がる→アンサンブルの分散は下がる\
バギングのほうがペースティングより良いモデルになることが多い

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf=BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500, 
    max_samples=100, bootstrap=True, n_jobs=-1)
bag_clf.fit(X_train, y_train)
y_pred=bag_clf.predict(X_test)

* OOB（out-of-bag)検証：訓練中に見られなかったインスタンスを使って検証すること
* ランダムパッチメソッド(random patch)：訓練インスタンスと特徴量の両方をサンプリングすること
* ランダムサブスペースメソッド（random subspace)：訓練インスタンスはすべて使い、特徴量だけサンプリングすること\
特徴量をサンプリングすると、予測器の多様性は上がり、わずかにバイアスが上がる分、分散を小さくすることができる。

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators = 500,
    bootstrap = True, n_jobs = -1, oob_score=True
)
bag_clf.fit(X_train, y_train)
bag_clf.oob_score_

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

ランダムフォレスト：決定木のアンサンブル\
バギングメソッド(たまにペースティング）で訓練される\
max_samples=訓練セットサイズ
ノードを分割するときに最良の特徴量を探すのではなく、特徴量の無作為なサブセットから最良の特徴量を探す\


In [ ]:
#random forestの言いかえ
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
    n_estimators  = 500, max_samples = 1.0, bootstrap = True, n_jobs = -1#ペースティングを使いたい場合はbootstrapをfalseに
)

Extra-Tree:Extremely Randomized Trees\
最良のしきい値を探すのではなく、個々の特徴量の閾値を無作為なものにする

普段の流れ：
* まず閾値の候補として、データの間の数を選ぶ
* それぞれの候補に関してジニ係数などを求めて、最適な閾値を選ぶ


その特徴量を使うノードが平均して不純度をどれくらい減らすかを調べることにより、特徴量の重要性を測る。

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators = 500, n_jobs = -1)#n_jobs 訓練、予測に使うCPUコアの数を指示
rnd_clf.fit(iris["data"], iris["target"])

In [ ]:
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

ブースティング(boosting):複数の弱学習器を結合して強学習器を作れるあらゆるアンサンブルメソッド
* AdaBoost
* 勾配ブースティング（gradient boosting)

AdaBoostの手順
* ベースの分類器を訓練し、訓練セットを対象として予測
* 分類に失敗した訓練真スタンスの相対的な重みを上げる
* 更新された重みを使って次の分類器を訓練し、訓練セットの予測をして、重みを更新\
これを繰り返し\
➡コスト関数を最小化するためにパラメータを操作するのではなく、予測器を追加してアンサンブルを改良していく
https://www.youtube.com/watch?v=LsK-xG1cLYA
1. 個々のインスタンスに駆けられた重み$w^{(i)}$は初期状態で$\frac{1}{m}$
2. 誤り率$r_1$が計算される$r_j = \frac{\Sigma_{i=1, \hat{y}_j^{(i)}\neq y^{(i)}}^{m}w^{(i)}}{\Sigma_{i=1}^m w^{(i)}}$
3. 予測器の重みが計算される➡正確なほど重みは高くなる$\alpha_j = \eta \log \frac{1-r_j}{r_j}$
4. インスタンスの重みを更新する。➡誤った分類をされたインスタンスの重みは大きくなる$for\:i = 1,2,...,m\:w^{(i)} = \left\{ \,
    \begin{aligned}
    & w^{(i)}　　　　　if  \hat{y}_j^{(i)}=y^{(i)}\\
    & w^{(i)}exp(\alpha_j)　if  \hat{y}_j^{(i)}\neq y^{(i)}
    \end{aligned}
\right.$
5. すべてのインスタンスの重みを正規化
6. すべての予測器の予測を計算し、予測器の重み$\alpha_j$を使って予測に重みを与えたうえで、重み付きの多数決で選ばれたクラスを予測結果とする。

決定株(decision stump):max_depth=1の決定木

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=10000, noise=0.25)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate = 0.5
)
ada_clf.fit(X_train, y_train)

勾配ブースティング:新予測器を前の予測器の残差（residual error)に適合させようとする。\
https://www.youtube.com/watch?v=3CC4N4z3GJc \
収縮(shrinkage)：learning_rateハイパーパラメータは個々の木の影響力を調整するが、これを低い値にすることによって、多くの決定木を追加しなけらばならないが、予測の汎化性能が上がる\
決定木の最適な数は、早期打ち切りを使えばわかる。


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt=GradientBoostingRegressor(max_depth=2, n_estimators = 120)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

In [ ]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)#既存の決定木を残す

min_val_error = float("inf")
error_going_up=0
for n_estimators in range(1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred=gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up=0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break

個々の決定木を訓練するために使われる訓練インスタンスの割合を指定→個々の決定木は無作為に選択された訓練インスタンスうを使って訓練される。\
➡バイアスを上げて分散を下げる\
これを確率的勾配ブースティング（stochastic gradient boosting)という

In [ ]:
import xgboost 

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [ ]:
xgb_reg.fit(X_train, y_train,
           eval_set = [(X_val, y_val)], early_stopping_rounds=2)
y_pred=xgb_reg.predict(X_val)

スタッキング(stacking,　スタック汎化：stacked generalization):アンサンブルに含まれるすべての予測器の予測を集計するところもモデルとして訓練しよう！\
ブレンダ：blender, メタ学習器：meta learner…最後の予測器のこと。最終的な予測を返す。
シンプルなスタッキング
1. 訓練セットを２つのサブセットに分割→１つは第１層の予測器の訓練
2. その予測器で第2セットの予測→この予測値はブレンダの入力
3. 第2セットの実際の値を訓練データとしてブレンダを訓練

実際は異なるブレンダを訓練して、第2層とする。
第3層を追加する。